# Testning av ml-modell

Testar att sätta upp ett flöde för användning av min AI-modell model_keyword_adder. 
Modellen är en prediktionsmodell som klassificerar sannolikheten för att ett keyword av många ska läggas till på en produkt.

Jag tittar också på olika threshold värden för de predikterade värdena. För att få en uppfattning om hur pass "säkra" de är att prediktera rätt.
Slutsatsen där är att den fungerar mycket bra som beslutsstöd för en ERFAREN berikare. Men för att ge ett säkert val för minst 1 keyword är den klart sämre. 
För att den ska vara pålitlig där behöver jag modellera mer och också testa med att inkludera fler Keywords av dem som jag tidigt i Databearbetningsfasen tog bort. 

Slutsats: Jag vill återvända till modelleringsfasen för att jobba vidare och ta fram fler varianter av modellen och testa dem mot varandra. Och jag gillar att jag även testade live. Mot ny data. 
Det blir ju snarare som ytterligare testdata där jag kan se de faktiska keywordsen.


## Inläsning och kontroll av ML-modell

In [67]:
import joblib
import pandas as pd
import numpy as np

In [68]:
kw_model = joblib.load("ml_models/keyword_adder_model_three.joblib")
kw_model_mapper = joblib.load("ml_models/keyword_adder_model_three_MLB.joblib")

print(type(kw_model))
print(type(kw_model_mapper))
print(kw_model.classes_)
print(kw_model_mapper.classes_)

<class 'sklearn.pipeline.Pipeline'>
<class 'sklearn.preprocessing._label.MultiLabelBinarizer'>
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194]
['100% Merinoull' '100% Ull' '13" datorfodral' '15" datorfodral'
 '3-säsongs sovsäckar' '3-säsongs tält' '3-säson

In [69]:
# Kollar närmare på vad modellen behöver för indata
kw_model.named_steps["preprocess"]

,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('text', ...), ('ipim', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and ``feature_name``.

In [70]:
# Skapar en Pandas Dataframe från exporterad fil.
raw_df = pd.read_csv("resources/Keyword_tester.csv", encoding="utf-8", sep=";")
raw_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   Product number              32 non-null     int64
 1   Product name                32 non-null     str  
 2   Online Status               32 non-null     str  
 3   Text status                 32 non-null     str  
 4   Headline                    32 non-null     str  
 5   Short Text                  32 non-null     str  
 6   Long Text                   30 non-null     str  
 7   SEO-Keyword                 32 non-null     str  
 8   iPIM Classification [iPIM]  32 non-null     str  
dtypes: int64(1), str(8)
memory usage: 2.4 KB


In [71]:
# Sätter upp en ny Pandas DataFrame inför prediktionerna

df = raw_df[["Product number", "Headline", "Short Text", "Long Text", "SEO-Keyword", "iPIM Classification [iPIM]"]]
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   Product number              32 non-null     int64
 1   Headline                    32 non-null     str  
 2   Short Text                  32 non-null     str  
 3   Long Text                   30 non-null     str  
 4   SEO-Keyword                 32 non-null     str  
 5   iPIM Classification [iPIM]  32 non-null     str  
dtypes: int64(1), str(5)
memory usage: 1.6 KB


In [72]:
# Droppar två rader där Long Text saknas
df = df.dropna()
df.info()

<class 'pandas.DataFrame'>
Index: 30 entries, 0 to 31
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   Product number              30 non-null     int64
 1   Headline                    30 non-null     str  
 2   Short Text                  30 non-null     str  
 3   Long Text                   30 non-null     str  
 4   SEO-Keyword                 30 non-null     str  
 5   iPIM Classification [iPIM]  30 non-null     str  
dtypes: int64(1), str(5)
memory usage: 1.6 KB


In [73]:
# Skapar ny kolumn för variabeln X_text_min
df["X_text_min"] = df["Headline"] + df["Short Text"] + df["Long Text"]
df.info()

<class 'pandas.DataFrame'>
Index: 30 entries, 0 to 31
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   Product number              30 non-null     int64
 1   Headline                    30 non-null     str  
 2   Short Text                  30 non-null     str  
 3   Long Text                   30 non-null     str  
 4   SEO-Keyword                 30 non-null     str  
 5   iPIM Classification [iPIM]  30 non-null     str  
 6   X_text_min                  30 non-null     str  
dtypes: int64(1), str(6)
memory usage: 1.9 KB


In [74]:
# Skickar text, ipim och seokw till modellen och spara topp 5 prediktioner i ny kolumn
X = df[['X_text_min', 'iPIM Classification [iPIM]', 'SEO-Keyword']]

# Genomför prediktioner
preds = kw_model.predict_proba(X)

# Hämtar de 5 bästa predikterade keywordsen och tillhörande sannolikheter för varje rad
predicted_keywords = []
predicted_values = []
for row in preds:
    top5_idx = np.argsort(row)[::-1][:5]
    top5_probs = row[top5_idx]
    top5_keywords = [kw_model_mapper.classes_[i] for i in top5_idx]
    predicted_keywords.append(top5_keywords)
    predicted_values.append(list(top5_probs))

# Hämtar det starkaste keywordet och tail keywords baserat på thresholds
top_keyword_threshold = []
keyword_two_to_five = []
top_threshold = 0.8
min_threshold = 0.25

for kws, probs in zip(predicted_keywords, predicted_values):
    # Top keyword endast om högre än top_threshold
    top_kw = kws[0] if probs[0] >= top_threshold else ""
    # Ta keywords 2-5 som ligger mellan min_threshold och top_threshold
    tail = [k for k, p in zip(kws[1:], probs[1:]) if min_threshold <= p < top_threshold]
    top_keyword_threshold.append(top_kw)
    keyword_two_to_five.append(tail)

# Spara till DataFrame
df['predicted keywords'] = predicted_keywords
df['predicted values'] = predicted_values
df['top keyword'] = top_keyword_threshold
df['tail keyword'] = keyword_two_to_five

In [75]:
df.info()

<class 'pandas.DataFrame'>
Index: 30 entries, 0 to 31
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Product number              30 non-null     int64 
 1   Headline                    30 non-null     str   
 2   Short Text                  30 non-null     str   
 3   Long Text                   30 non-null     str   
 4   SEO-Keyword                 30 non-null     str   
 5   iPIM Classification [iPIM]  30 non-null     str   
 6   X_text_min                  30 non-null     str   
 7   predicted keywords          30 non-null     object
 8   predicted values            30 non-null     object
 9   top keyword                 30 non-null     str   
 10  tail keyword                30 non-null     object
dtypes: int64(1), object(3), str(7)
memory usage: 2.8+ KB


In [76]:
df[["Short Text", "SEO-Keyword", "top keyword", "tail keyword", "predicted keywords", "predicted values"]].head(30)

,Short Text,SEO-Keyword,top keyword,tail keyword,predicted keywords,predicted values
0,"<p>Rak, tidlös byxmodell som passar bra såväl ...",Klätterbyxor,,[Byxor för sportklättring],"[Vardagsbyxor, Byxor för sportklättring, Bergk...","[0.5490588678298581, 0.5489890856041676, 0.100..."
1,<p>Lätta shorts med fyrvägsstretch som passar ...,Klätterbyxor,,[Vardagsbyxor],"[Byxor för sportklättring, Vardagsbyxor, Byxor...","[0.48393659427592645, 0.42309637666874955, 0.0..."
2,<p>Lätt och snabbtorkande väst med ventilerand...,Väst,,[Syntetisolerade],"[Fodrade västar, Syntetisolerade, Ej isolerad,...","[0.788558583076752, 0.4432441572075828, 0.0823..."
3,"<p>Mångsidiga vandringsshorts i lätt, stretchi...",Vandringsbyxor,,[Vandringsbyxor för sommar],"[Byxor för resa, Vandringsbyxor för sommar, 3-...","[0.563375187976591, 0.4583069522549747, 0.2168..."
4,<p>Lätta och stretchiga vandringsbyxor för bar...,Vandringsbyxor,,[3-säsongs vandringsbyxor],"[Vandringsbyxor för sommar, 3-säsongs vandring...","[0.5031209323266682, 0.4524171083588464, 0.192..."
6,<p>Snabbtorkande och stretchig t-shirt för akt...,Funktionstopp,Funktionströjor,[Vardags-T-shirts],"[Funktionströjor, Vardags-T-shirts, Kläder i m...","[0.9459657863738886, 0.32104130498672834, 0.08..."
7,<p>Lätta och snabbtorkande shorts - perfekt fö...,Badbyxor,,[],"[Shorts, Vardagsbyxor, Syntetiskt material, UV...","[0.26747553814180597, 0.12570181924784715, 0.1..."
8,<p>Lätt vindjacka i återvunnen nylon som skydd...,Vindjacka,,[],"[Vindjacka, Vår-och höstjackor, Softshell jack...","[0.7574560777516728, 0.23806096113115527, 0.14..."
9,<p>Lätt och snabbtorkande t-shirt i mjuk polye...,T-shirt,,[],"[Vardags-T-shirts, Funktionströjor, 100% Merin...","[0.7779605238057177, 0.2410805139221658, 0.048..."
10,<p>Ultralätt och ventilerande keps för löpning...,Keps,Kepsar,[],"[Kepsar, Sommarhattar, Truckerkepsar, Vintermö...","[0.8996913512549041, 0.2360483615330701, 0.105..."
